In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.cross_validation import cross_val_score, train_test_split

C:\Users\Siddh\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [219]:
data = pd.read_excel('Folds5x2_pp.xlsx')

In [220]:
data.columns

Index(['AT', 'V', 'AP', 'RH', 'PE'], dtype='object')

In [221]:
data.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [222]:
data.columns = ['AT', 'V', 'AP', 'RH', 'Target']

In [223]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,0:4], data.iloc[:,4], test_size=0.2)
X_train_mean = X_train.mean()
X_train_sd = X_train.std()
X_train_std = (X_train - X_train_mean) / X_train_sd
X_test_std = (X_test - X_train_mean) / X_train_sd
y_train_mean = y_train.mean()
y_train_sd = y_train.std()
y_train_std = (y_train - y_train_mean) / y_train_sd
y_test_std = (y_test - y_train_mean) / y_train_sd

In [75]:
data_scaled = data.iloc[:,0:4]/data.iloc[:,0:4].std() - data.iloc[:,0:4].mean()/data.iloc[:,0:4].std()

In [82]:
data_scaled['Target'] = data.iloc[:,4]

In [85]:
data_scaled.head()

,AT,V,AP,RH,Target
0,-0.629486,-0.987245,1.820393,-0.009519,463.26
1,0.741870,0.681010,1.141803,-0.974570,444.37
2,-1.951195,-1.172956,-0.185068,1.289772,488.56
3,0.162197,0.237191,-0.508366,0.228148,446.48
4,-1.185007,-1.322470,-0.678435,1.596616,473.90


In [90]:
#Activation Function MSE
def activation_function(Target, features, coefficients):
    return ((Target - np.dot(features, coefficients))**2).mean()

In [89]:
# Predictor function
def predictor(features, coefficients):
    return np.dot(features, coefficients)

In [229]:
def linear_regressor(features, dep_variable, alpha = 0.05, iterations = 100):
    intercept = np.ones((features.shape[0],1))
    features = np.column_stack((intercept, features))
    coeff = np.zeros(features.shape[1])
    for i in range(iterations):
        coeff[0] = coeff[0] + 2*(dep_variable-np.dot(features, coeff)).mean()*alpha
        coeff[1] = coeff[1] + 2*((dep_variable-np.dot(features, coeff))*features[:,1]).mean()*alpha
        coeff[2] = coeff[2] + 2*((dep_variable-np.dot(features, coeff))*features[:,2]).mean()*alpha
        coeff[3] = coeff[3] + 2*((dep_variable-np.dot(features, coeff))*features[:,3]).mean()*alpha
        coeff[4] = coeff[4] + 2*((dep_variable-np.dot(features, coeff))*features[:,4]).mean()*alpha
    return coeff
                           
                           
    

In [230]:
result = linear_regressor(X_train_std, y_train_std, alpha = 0.01, iterations = 100)

In [231]:
result

array([-1.93633362e-14, -4.79535776e-01, -3.85838498e-01,  1.34337530e-01,
        4.08954097e-02])

In [232]:
np.dot(np.column_stack((np.ones(len(X_train)),X_train)),result)

array([ 95.51860994, 117.13926918, 115.85373989, ..., 103.21111632,
       100.38620658, 113.22263332])

In [233]:
np.column_stack((np.ones(len(data)), data.iloc[:,0:4]))

array([[1.00000e+00, 1.49600e+01, 4.17600e+01, 1.02407e+03, 7.31700e+01],
       [1.00000e+00, 2.51800e+01, 6.29600e+01, 1.02004e+03, 5.90800e+01],
       [1.00000e+00, 5.11000e+00, 3.94000e+01, 1.01216e+03, 9.21400e+01],
       ...,
       [1.00000e+00, 3.13200e+01, 7.43300e+01, 1.01292e+03, 3.64800e+01],
       [1.00000e+00, 2.44800e+01, 6.94500e+01, 1.01386e+03, 6.23900e+01],
       [1.00000e+00, 2.16000e+01, 6.25200e+01, 1.01723e+03, 6.78700e+01]])

In [235]:
def root_mse(y_pred, y_test):
    sse = (y_test - y_pred) ** 2
    mse = sse.mean()
    return np.sqrt(mse)

In [236]:
y_train.shape

(7654,)

In [237]:
data.iloc[:,0:4].shape

(9568, 4)

In [238]:
intercept = np.ones((X_train_std.shape[0],1))
features = np.column_stack((intercept, X_train_std))

y_pred = np.dot(features, result)
print('Train Set RMSE: {0}'.format(root_mse(y_pred, y_train_std)))

Train Set RMSE: 0.3171660462534026
